# <font color='black'>Регрессионный анализ социально-экономических процессов, 2025 - 2026 </font>
## <font color='black'> Практическое занятие 6. Гребневая регрессия. Меры качества линейной регрессионной модели. Сравнение альтернативных спецификаций моделей </font>

Краткое описание данных:

* lngdp2 - логарифм ВВП на душу населения - зависимая переменная

В качестве предикторов выступают 6 индексов качества гос. управления WGI (Worldwide Governance Indicators)
* va - voice and accountability
* rl - rule and law
* rq - regulatory quality
* gove - government effectiveness
* ps - political stability
* cc - control of corruption

Распределения этих индексов приведено в рамках проекта к стандартному нормальному, поэтому мы не используем дополнительно стандартизацию

In [34]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from statsmodels.stats.anova import anova_lm

In [35]:
dta = pd.read_stata('data_ridge.dta')
dta = dta.dropna()

Оценим модель m1 на исходных данных. Можем ли мы доверять полученным результатам?

In [36]:
m1 = smf.ols(formula = "lngdp2 ~ va + rl + rq + gove + ps + cc", data = dta).fit()
print(m1.summary())

                            OLS Regression Results                            
Dep. Variable:                 lngdp2   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.645
Method:                 Least Squares   F-statistic:                     51.81
Date:                Sun, 18 Jan 2026   Prob (F-statistic):           3.09e-35
Time:                        21:25:30   Log-Likelihood:                -191.70
No. Observations:                 169   AIC:                             397.4
Df Residuals:                     162   BIC:                             419.3
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.6997      0.060    145.828      0.0

Выведем корреляционную матрицу для наших предикторов:

In [37]:
dta[["va", "rl", "rq", "gove", "ps", "cc"]].corr().round(3)

,va,rl,rq,gove,ps,cc
va,1.000,0.774,0.788,0.751,0.651,0.762
rl,0.774,1.000,0.890,0.943,0.770,0.952
rq,0.788,0.890,1.000,0.943,0.647,0.877
gove,0.751,0.943,0.943,1.000,0.687,0.940
ps,0.651,0.770,0.647,0.687,1.000,0.711
cc,0.762,0.952,0.877,0.940,0.711,1.000


Кроме этого, понять, есть ли у нас свидетельства в пользу сильной мультиколлинеарности, нам помогут коэффициенты VIF (коэффициенты "вздутия" дисперсии). Проинтерпретируйте полученные результаты

In [38]:
X = dta[["va", "rl", "rq", "gove", "ps", "cc"]]
X = add_constant(X)

vif_data = pd.DataFrame({'variables':X.columns[1:], 'VIF':[variance_inflation_factor(X.values, i+1) for i in range(len(X.columns[1:]))]})
print(vif_data)

  variables        VIF
0        va   3.099573
1        rl  17.089986
2        rq  10.764995
3      gove  20.736053
4        ps   2.629430
5        cc  13.327886


 Применим гребневую регрессию для получения более устойчивых результатов. Воспользуемся процедурой кросс-валидации для подбора оптимального параметра регуляризации. Разделим наш массив на 2 подвыборки: тестовую (20% данных) и обучающую (соответственно, 80% данных)

In [39]:
train, test = train_test_split(dta, test_size = 0.2, random_state = 1)

Для начала используем 1 в качестве параметра регуляризации и выведем оценки коэффициентов в модели гребневой регрессии:

In [40]:
ridge1 = Ridge(alpha = 1)
ridge1.fit(train[["va", "rl", "rq", "gove", "ps", "cc"]], train["lngdp2"])
ridge1.coef_

array([-0.31415719,  0.04413839,  0.3977781 ,  0.94014374,  0.34709847,
       -0.28108621])

Напишем функцию для расчета стандартных ошибок:

In [41]:
def ridge_se(model, X, y):
    n, p = X.shape
    X_matrix = X.values if hasattr(X, 'values') else X

    y_pred = model.predict(X)
    residuals = y - y_pred
    sigma_sq = np.sum(residuals**2) / (n - p - 1)

    XTX = X_matrix.T @ X_matrix
    lambda_I = model.alpha * np.eye(p)
    inv_matrix = np.linalg.inv(XTX + lambda_I)

    cov_matrix = sigma_sq * inv_matrix @ XTX @ inv_matrix
    se = np.sqrt(np.diag(cov_matrix))

    return se

Рассчитаем стандартные ошибки для оценок коэффициентов нашей первой модели гребневой регрессии:

In [42]:
X = train[["va", "rl", "rq", "gove", "ps", "cc"]]
y = train['lngdp2']
se1 = ridge_se(ridge1, X, y)

se1

array([0.12421295, 0.24570921, 0.20372793, 0.25520473, 0.11643805,
       0.21791001])

Для удобства представим ниже таблицу, в которой сравним оценки коэффициентов и их значимость в исходной модели и модели гребневой регрессии (с параметром $\alpha$ = 1):

In [43]:
ridge1_data = pd.DataFrame({'variables':X.columns, 'coef': m1.params[1:], 'se': m1.bse[1:], 't': m1.params[1:]/m1.bse[1:],
                            'coef_ridge':ridge1.coef_, 'se_ridge': se1, 't_ridge': ridge1.coef_/se1})
print(ridge1_data)

     variables      coef        se         t  coef_ridge  se_ridge   t_ridge
va          va -0.175811  0.108381 -1.622160   -0.314157  0.124213 -2.529182
rl          rl -0.064419  0.253270 -0.254350    0.044138  0.245709  0.179637
rq          rq  0.300197  0.205736  1.459135    0.397778  0.203728  1.952497
gove      gove  1.067160  0.274419  3.888790    0.940144  0.255205  3.683881
ps          ps  0.265687  0.103624  2.563944    0.347098  0.116438  2.980971
cc          cc -0.296014  0.218021 -1.357736   -0.281086  0.217910 -1.289919


Подберем оптимальный параметр регуляризации. Для этого используем k-блочную кросс-валидацию: то есть, массив разбивается на k равных подвыборок, далее проводим для каждого заданного параметра $\alpha$ k итераций: j-ая подвыборка выступает тестовой, остальные подвыборки составляют обучающую. Считаем среднее MSE по k итерациям для каждого значения $\alpha$ и далее останавливаемся на том значении $\alpha$, при котором усредненное MSE принимает минимальное значение

In [44]:
alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

grid_search = GridSearchCV(ridge1, {'alpha': alphas}, cv = 5)
grid_search.fit(train[["va", "rl", "rq", "gove", "ps", "cc"]], train["lngdp2"])

print("Best Regularization Parameter:", grid_search.best_params_)

Best Regularization Parameter: {'alpha': 10}


In [45]:
ridge2 = Ridge(alpha = 10)
ridge2.fit(train[["va", "rl", "rq", "gove", "ps", "cc"]], train["lngdp2"])
ridge2.coef_

array([-0.22752006,  0.12273421,  0.40647171,  0.57121629,  0.28564373,
       -0.04623492])

In [46]:
se2 = ridge_se(ridge2, X, y)

se2

array([0.09876988, 0.11202828, 0.11085828, 0.10739525, 0.09432917,
       0.11082235])

In [47]:
ridge2_data = pd.DataFrame({'variables':X.columns, 'coef': m1.params[1:], 'se': m1.bse[1:], 't': m1.params[1:]/m1.bse[1:],
                            'coef_ridge':ridge2.coef_, 'se_ridge': se2, 't_ridge': ridge2.coef_/se2})
print(ridge2_data)

     variables      coef        se         t  coef_ridge  se_ridge   t_ridge
va          va -0.175811  0.108381 -1.622160   -0.227520  0.098770 -2.303537
rl          rl -0.064419  0.253270 -0.254350    0.122734  0.112028  1.095565
rq          rq  0.300197  0.205736  1.459135    0.406472  0.110858  3.666588
gove      gove  1.067160  0.274419  3.888790    0.571216  0.107395  5.318823
ps          ps  0.265687  0.103624  2.563944    0.285644  0.094329  3.028159
cc          cc -0.296014  0.218021 -1.357736   -0.046235  0.110822 -0.417199


Далее для понимания, можем ли мы обобщать результаты на более широкую выборку, сравним $R^2$ и $MSE$ на тестовой и обучающей подвыборках:

In [48]:
y_pred_test = ridge2.predict(test[["va", "rl", "rq", "gove", "ps", "cc"]])
mse_test = mean_squared_error(test["lngdp2"], y_pred_test)

In [49]:
r2_test = r2_score(test["lngdp2"], y_pred_test)

In [50]:
y_pred_train = ridge2.predict(train[["va", "rl", "rq", "gove", "ps", "cc"]])
mse_train = mean_squared_error(train["lngdp2"], y_pred_train)

In [51]:
r2_train = r2_score(train["lngdp2"], y_pred_train)

In [52]:
print(f'R2 обучающая выборка: {r2_train:.3f}\nR2 тестовая выборка: {r2_test:.3f}')
print(f'MSE обучающая выборка: {mse_train:.3f}\nMSE тестовая выборка: {mse_test:.3f}')

R2 обучающая выборка: 0.639
R2 тестовая выборка: 0.683
MSE обучающая выборка: 0.637
MSE тестовая выборка: 0.373


Оценим две спецификации модели, вложенные друг в друга - они будут различаться на один параметр. Проследим, как изменяются меры коэффициента детерминации и скорректированного коэффициента детерминации с добавлением предиктора.

$R^2$ скорректированный ($R^2_{adj}$) штрафует модель за "нагруженность", то есть, за дополнительные предикторы. $R^2_{adj}$ рассчитывается по следующей формуле:

$R^2_{adj} = 1 - \frac{RSS \times (N-1)}{TSS \times (N-k-1)}$, где k - это количество предикторов в модели

In [53]:
m2 = smf.ols(formula = "lngdp2 ~ rl", data = dta).fit(cov_type = "HC3")
print(m2.summary())

                            OLS Regression Results                            
Dep. Variable:                 lngdp2   R-squared:                       0.553
Model:                            OLS   Adj. R-squared:                  0.551
Method:                 Least Squares   F-statistic:                     288.2
Date:                Sun, 18 Jan 2026   Prob (F-statistic):           3.36e-38
Time:                        21:25:30   Log-Likelihood:                -214.12
No. Observations:                 169   AIC:                             432.2
Df Residuals:                     167   BIC:                             438.5
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.7401      0.065    134.345      0.0

In [54]:
m3 = smf.ols(formula = "lngdp2 ~ rl + va", data = dta).fit(cov_type = "HC3")
print(m3.summary())

                            OLS Regression Results                            
Dep. Variable:                 lngdp2   R-squared:                       0.554
Model:                            OLS   Adj. R-squared:                  0.548
Method:                 Least Squares   F-statistic:                     144.3
Date:                Sun, 18 Jan 2026   Prob (F-statistic):           4.78e-37
Time:                        21:25:30   Log-Likelihood:                -214.08
No. Observations:                 169   AIC:                             434.2
Df Residuals:                     166   BIC:                             443.6
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.7406      0.066    132.641      0.0

In [55]:
print("Model2 R-squared:", m2.rsquared.round(3), "Model2 R-squared adjusted:", m2.rsquared_adj.round(3))
print("Model3 R-squared:", m3.rsquared.round(3), "Model3 R-squared adjusted:", m3.rsquared_adj.round(3))

Model2 R-squared: 0.553 Model2 R-squared adjusted: 0.551
Model3 R-squared: 0.554 Model3 R-squared adjusted: 0.548


В то время как $R^2$ увеличивается с добавлением новых предикторов, $R^2_{adj}$ может и уменьшиться (в случае добавленных незначимых предикторов). Для лучшего понимания выведем отдельно таблицу разложения вариации для m3.

In [56]:
print(anova_lm(m3))

             df      sum_sq     mean_sq           F        PR(>F)
rl          1.0  154.495711  154.495711  205.737960  7.193484e-31
va          1.0    0.048890    0.048890    0.065106  7.989169e-01
Residual  166.0  124.655110    0.750934         NaN           NaN


Важно понять, устойчивы ли показатели качества регрессионной модели. Для того, чтобы результаты были более обоснованы и не опирались лишь на одно разбиение массива на тестовую и обучающую выборки, используем k-блочную кросс-валидацию (поделим данные на 5 фолдов, проведем 5 итераций и усредним полученные результаты: на каждой итерации алгоритма модель обучается на 4 фолдах, тестируется на оставшейся 5-ой части выборки).

Как мы видим, результаты неустойчивы, что вполне объяснимо с учетом наших данных и спецификации модели

In [57]:
X = dta[["rl", "va"]]
y = dta["lngdp2"]

In [58]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
R2 = []

In [59]:
for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    m1.cv = LinearRegression()
    m1.cv.fit(X_train, y_train)

    y_pred = m1.cv.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    R2.append(r2)

    print("Fold", fold+1, "R2:", r2)

average_R2 = sum(R2) / len(R2)
print("Average R2:", average_R2)

Fold 1 R2: 0.6590238366435552
Fold 2 R2: 0.4294918805308041
Fold 3 R2: 0.536419324298262
Fold 4 R2: 0.6813853876487733
Fold 5 R2: 0.28559785188443554
Average R2: 0.5183836562011661


Сравним альтернативные спецификации моделей m3 и m4 при помощи информационных критериев AIC и BIC.

In [60]:
m4 = smf.ols(formula = "lngdp2 ~ va + gove", data = dta).fit(cov_type = "HC3")
print(m4.summary())

                            OLS Regression Results                            
Dep. Variable:                 lngdp2   R-squared:                       0.634
Model:                            OLS   Adj. R-squared:                  0.629
Method:                 Least Squares   F-statistic:                     180.3
Date:                Sun, 18 Jan 2026   Prob (F-statistic):           2.43e-42
Time:                        21:25:30   Log-Likelihood:                -197.34
No. Observations:                 169   AIC:                             400.7
Df Residuals:                     166   BIC:                             410.1
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.6999      0.060    144.622      0.0

In [61]:
aic_m3 = m3.aic
aic_m4 = m4.aic

bic_m3 = m3.bic
bic_m4 = m4.bic

print("Model3 AIC:", aic_m3.round(3), "Model4 AIC:", aic_m4.round(3))
print("Model3 BIC:", bic_m3.round(3), "Model4 BIC:", bic_m4.round(3))

Model3 AIC: 434.166 Model4 AIC: 400.684
Model3 BIC: 443.556 Model4 BIC: 410.074


In [62]:
p = len(m3.params)
LL = m3.llf
aicm3 = 2*p - 2*LL
aicm3

np.float64(434.1664274971729)

In [63]:
bicm3 = np.log(len(dta))*p - 2*LL
bicm3

np.float64(443.5561236419421)

Для вложенных моделей мы можем использовать F-test. Статистика для этого теста рассчитывается следующим образом:

$F = \frac{(RSS_{short} - RSS_{long})/Δ df}{RSS_{long}/df_{long}}$

При верной нулевой гипотезе такая статистика имеет распределение Фишера с количеством степеней свободы: $df_{1} = \Delta df$, $df_{2} = df_{long}$

In [64]:
anovaResults = anova_lm(m2, m3)
print(anovaResults)

   df_resid        ssr  df_diff  ss_diff         F    Pr(>F)
0     167.0  124.70400      0.0      NaN       NaN       NaN
1     166.0  124.65511      1.0  0.04889  0.065106  0.798917
